# 1 - Web Scraping /Scrapeo (Bs4)
<p align = 'center'>
<img src = 'https://editor.analyticsvidhya.com/uploads/75676cover.jfif'/>
</p>

Web scraping o raspado web, es una técnica utilizada mediante programas de software para extraer información de sitios web. Usualmente, estos programas simulan la navegación de un humano en la web ya sea utilizando el protocolo HTTP manualmente, o incrustando un navegador en una aplicación.

El web scraping está muy relacionado con la indexación de la web, la cual indexa la información de la web utilizando un robot y es una técnica universal adoptada por la mayoría de los motores de búsqueda. Sin embargo, el web scraping se enfoca más en la transformación de datos sin estructura en la web, como el formato HTML, en datos estructurados que pueden ser almacenados y analizados en una base de datos central, en una hoja de cálculo o en alguna otra fuente de almacenamiento. Alguno de los usos del web scraping son la comparación de precios en tiendas, la monitorización de datos relacionados con el clima de cierta región, la detección de cambios en sitios webs y la integración de datos en sitios webs. 

En los últimos años el web scraping se ha convertido en una técnica muy utilizada dentro del sector del posicionamiento web gracias a su capacidad de generar grandes cantidades de datos para crear contenidos de calidad.

Podríamos pensar que el web scraping es nuestro recurso a falta de una API o un feed RSS. A falta de una fuente de datos, siempre podemos extraer aquello que sale por pantalla.

### Extracción desde el HTML

Para scrapear necesitamos saber que pinta tiene la **estructura general** que tiene un HTML.

El HTML consiste en contenido `<etiquetado>`, es como si fueran cajas de contenido, organizado de manera jerárquica:

```
<html>
    <head>
        <title>Titulo de la pagina</title>
    </head>
    <body>
        <h1>Cabecera</h1>
        <p>Parrafo</p>
    </body>
</html>
```

$$$$

Las etiquetas el HTML se pueden clasificar en varios grupos, dependiendo del tipo de contenido que posea. Estos son algunos ejemplos:

+ cabecera: `<h1>`, `<h2>`, `<h3>`, `<hgroup>`...
+ texto: `<b>`, `<p>`, `<span>`...
+ embebido: `<audio>`, `<img>`, `<video>`...
+ tabular: `<table>`, `<tr>`, `<td>`, `<tbody>`...
+ secciones: `<header>`, `<section>`, `<article>`...
+ metadata: `<meta>`, `<title>`, `<script>`...

$$$$


Las etiquetas pueden tener atributos. Por ejemplo:
 
`<div class="text-monospace" id="name_132", href="www.example.com"> Contenido de la pagina </div>` 

Esta etiqueta `div` tiene los siguientes atributos:

+ class: atributo con valor "text-monospace". La clase no es única en la página, varios elementos pueden tener la misma clase.
+ id: atributo con valor "name_132". El id de una etiqueta la identifica de manera unívoca, no puede haber dos etiquetas con el mismo id.
+ href: atributo con valor "www.example.com". El href suele contener el link a otra parte de la página.

Siguiendo con la analogía de las cajas, si una etiqueta de HTML es una caja, los atributos serían las pegatinas pegadas en la tapa de la caja.

Conociendo cual es el contenido que queremos extraer, debemos encontrar las etiquetas que nos interesan dentro de todo el HTML de la página web.

Usaremos la herramienta **[BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)**.

In [ ]:
#%pip install beautifulsoup4

In [ ]:
import requests as req

from bs4 import BeautifulSoup as bs #este alias es standard

### WIKIPEDIA

**[Países europeos según esperanza de vida](https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy)**

In [ ]:
url='https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy' #url de la pagina a scrapear

html = req.get(url).content #obtenemos el contenido de la pagina
html[:1000] #mostramos los primeros 1000 caracteres

In [ ]:
str(html).split('/head>')[1][:1000] #mostramos los primeros 1000 caracteres de la seccion body / Esto es una gualtrapada para ver el codigo html de la pagina

In [ ]:
soup = bs(html, 'html.parser') #creamos el objeto soup, es un nombre standar que se la da a este objeto

type(soup)

In [ ]:
# Tabla
tabla = soup.find('table') #buscamos la tabla, en este caso solo hay una. Si hubiera mas, solo nos devolveria la primera. Si queremos todas, usamos find_all
#soup.find_all('table')
type(tabla)

In [ ]:
tabla

In [ ]:
# Filas de la tabla
filas = tabla.find_all('tr')
filas

In [ ]:
#vamos a sacar el texto de cada fila solo para los 2 primeros elementos
for f in filas[:3]:
    print(type(f.text),f.text)
    
    print('-----------------')

In [ ]:
filas_nuevas = [f.text.strip().split('\n') for f in filas]

filas_nuevas[:5]

In [ ]:
# La primera fila es la cabecera, y hay que limpiar los espacios en blanco

cabecera=filas_nuevas[0]
cabecera

In [ ]:
#Vamos a arreglarlo

final=[] #aqui vamos a guardar el resultado final limpio

for fila in filas_nuevas:
    tmp=[]
    for caracter in fila:
        if caracter != '':
            tmp.append(caracter)
    final.append(tmp)
    
final[:5]

In [ ]:
# Nos hacemos un dataframe
import pandas as pd

nombres_columnas = final[0] #la primera fila es la cabecera

data=final[2:]

df = pd.DataFrame(data, columns=nombres_columnas)

df.head(10)

In [ ]:
NOS HEMOS QUEDADO AQUI

### Ejemplo geolocalización por IP

https://tools.keycdn.com/geo

**¿Dónde estoy?**

In [ ]:
url='https://tools.keycdn.com/geo'

In [ ]:
html=req.get(url).content

soup=bs(html, 'html.parser')

In [ ]:
soup.find('div', {'id': 'geoResult'})

In [ ]:
tabla=soup.find('div', {'id': 'geoResult'})#Puedo buscar por elemento, pero ademas precisando el ID

tabla.find_all('dd', {'class': 'col-8 text-monospace'}) #Elemento y añado tb la clase

In [ ]:
conexion=[e.text for e in tabla.find_all('dd', {'class': 'col-8 text-monospace'})]

conexion

In [ ]:
keys = [e.text for e in tabla.find_all('dt')]
keys

In [ ]:
dict = {k:v for k,v in zip(keys,conexion)} #me hago un diccionario con los datos de la conexion 

In [ ]:
dict

**Búsqueda según IP**

https://tools.keycdn.com/geo?host=137.255.90.7

In [ ]:
url='https://tools.keycdn.com/geo?host=137.255.90.7' #tengo parámetros en la direccion en funcion de la IP que busque...🤔

html=req.get(url).content

soup=bs(html, 'html.parser')

In [ ]:
tabla=soup.find('div', {'id': 'geoResult'})

tabla.find_all('dd', {'class': 'col-8 text-monospace'})

In [ ]:
tabla.find_all('dt')

In [ ]:
#'{:2.2f}'.format(12.5436363636363)  # formato en strings de numeros

In [ ]:
list_ip=['137.255.90.7', '255.255.90.7', '177.255.21.7']

In [ ]:
def geo(ip):
    
    url=f'https://tools.keycdn.com/geo?host={ip}' #Dinamica en fucnion de la IP que quiero buscar
    
    html=req.get(url).content

    soup=bs(html, 'html.parser')
    
    tabla=soup.find('div', {'id': 'geoResult'})
    
    conexion=[e.text for e in tabla.find_all('dd', {'class': 'col-8 text-monospace'})]

    return conexion

In [ ]:
for ip in list_ip:
    print(f'{geo(ip)}\n') #Le meto salto de linea para que lo veamos mas claro


### Ejemplo LinkedIn

In [ ]:
#Vamos intentar scrapear Linkedin
URL='https://www.linkedin.com/jobs/search/' #Esta es la direccion de la pagina de busqueda de empleo de Linkedin general


A esta direccion le podemos poner parametros para afinar nuestra búsqueda.

URL = 'https://www.linkedin.com/jobs/search/?keywords=data&location=Espa%C3%B1a&refresh=true'

`?keywords=data` nos dice que la palabra clave que queremos buscar es 'data'
    
`&location=Madrid` nos dice que queremos buscar en Madrid
    
`&refresh=true` nos dice que queremos que nos devuelva los resultados más recientes

Podemos filtrar mas nuestra búsqueda en la web y veremos que van apareciendo más ***parametros*** en la URL.

`&f_TPR=r120960` nos dice que empleos de la ultima semana. El parámetro va en segundos así que 60 * 60 * 24 * número de días atrás que queremos buscar

`&start={i*25}` donde i sería el número de página


In [ ]:
#Vamos a intentar scrapear esta URL
url = 'https://www.linkedin.com/jobs/search/?keywords=data&location=Madrid&f_TPR=r1296000&F_E=1&start=50'

#Estamos buscando para Madrid, termino data, ofertas publicadas en los ultimos 7 días y que nos muestre la segunda página de resultados

soup=bs(req.get(url).content, 'html.parser')

In [ ]:
# Intentad sacar los ingredientes de la recete de esta sopa y obtener para cada oferta de trabajo:

# - Titulo
# - Empresa
# - Ubicacion
# - Link de la empresa
# - Link de la oferta
# - Fecha de publicacion

In [ ]:
#Os recomiendo ir echando una ojeada al html y la sopa poco a poco para ver como esta estructurado
#Lo primero que tendremos que buscar será el elemento que contiene todas las ofertas....🤔

#PISTA -- base-search-card__info
#De aqui en adelante vosotros solos, tened presente que querremos guardar toda esa info en un dataframe

In [ ]:
def suma(*args):
    return sum(args)

suma(2, 2, 3, 45, 67, 890)

### REPASO FUNCIONES

In [ ]:
# repaso funciones

def suma(*args):
    return sum(args)

suma(2, 2, 3, 45, 67, 890)

In [ ]:
def saludar(nombre, lang='es', colega=True):
    s=''
    
    if colega:
        s='colega!!!'
        
    if lang=='es':
        print('Hola {} {}'.format(nombre, s))
        
    else:
        print('Hello {} buddy!!!'.format(nombre))

In [ ]:
def saludar_multiple(*lst, lang='es', colega=True):
    for e in lst:
        saludar(e, lang, colega)

In [ ]:
saludar('Pepe')

In [ ]:
saludar(['Pepe', 'en'])

In [ ]:
saludar(*['Pepe', 'en'])

In [ ]:
saludar_multiple('Ana', 'Pepe', 'Juan', 'Maria', lang='en')

In [ ]:
nombres=['Ana', 'Pepe', 'Juan', 'Maria']

config={'lang': 'es', 'colega': True}

In [ ]:
saludar_multiple(*nombres, **config)

In [ ]:
def function (*args, **kwargs):
    return 

### Nos atrevemos a meter nuestro código en una función que acepte parámetros para añadir a la url de base de linkedin?

### Metiendo todo en una funcion para que sea mas facil de usar

In [ ]:
import pandas as pd

def linkedin(num_pages, keywords, country, n_secs = 30000, exp = 1):
    
    URL='https://www.linkedin.com/jobs/search/'

    data=[]

    for i in range(num_pages):

        scrape_url=''.join([
            URL,  # url base
            f'?keywords={keywords}',   # palabras clave de busqueda
            f'&location={country}',   # pais lugar
            
            f'&f_TPR=r{n_secs}',        # segundos atras
            f'&F_E={exp}',            # experiencia (1,2,3)
            f'&start={i*25}'           # numero de pagina (i)
        ])


        html=req.get(scrape_url).content  # el html de la pagina

        soup=bs(html, 'html.parser')       # la sopa parseada

        for oferta in soup.find_all('div', 
                               class_ = "base-search-card__info"):
                               #class_="base-card base-card--link base-search-card base-search-card--link job-search-card"):
            # bucle para las ofertas
            titulo = oferta.find('h3', class_="base-search-card__title").text.strip()   # titulo de la ofertabase
            # titulo = oferta.find('span', class_="screen-reader-text").text.strip()   # titulo de la oferta

            empresa = oferta.find('h4', class_="base-search-card__subtitle").text.strip()   # nombre de la compañia
            
            link_comp = oferta.find('a', class_="hidden-nested-link").attrs['href']  # link de la compañia

            lugar = oferta.find('span', class_="job-search-card__location").text.strip()  # lugar

            link_ofer = oferta.find('a', class_="hidden-nested-link").attrs['href']   # link de la oferta

            fecha=oferta.find('time').attrs['datetime']       # fecha de publicacion

            data.append({
                'title': titulo,
                'name': empresa,
                'link de la compañia': link_comp,
                'location': lugar,
                'link de la oferta': link_ofer,
                'datetime': fecha
            })
    return pd.DataFrame(data)       
    #return (pd.DataFrame(data), scrape_url)


### Soporte

In [ ]:
#Ahora que tenemos la sopa parseada, vamos a buscar los elementos que nos interesan que son los que tienen la clase 'base-card base-card--link base-search-card base-search-card--link job-search-card'
lista_ofertas = soup.find('div', class_="base-search-card__info")
lista_ofertas
#soup.find_all('div', class_="base-search-card--link job-search-card")